In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from thermoextrap import *
import thermoextrap


In [2]:
# xtrapy stuff:
# note, just using single module right now.  Can retinker
import thermoextrap.xtrapy.core as core
import thermoextrap.xtrapy.xpan_beta as xpan_beta
import xarray as xr

In [3]:
from importlib import reload

In [4]:
order = 6
fs = [thermoextrap.symDerivAvgX(i) for i in range(order+1)]

In [5]:
# test Data
# samples
n = int(1e5)
# number of x values
nv = 5

_u_offset = 0.0
_x_offset = 0.0


np.random.seed(0)
u = np.random.rand(n) + _u_offset
x = np.random.rand(n, nv) + _x_offset

# second set, to testing
ub = np.random.rand(n) + _u_offset
xb = np.random.rand(n, nv) + _x_offset

# comparison of xtrapy

## Data averages

In [6]:
# Test "coefs"
ufunc, xufunc = thermoextrap.buildAvgFuncs(x, u, order)
coefs_list = [fs[i](ufunc, xufunc) for i in range(order+1)]

In [7]:
# coefficient class handler
# raw moments

# coefficient builder
s = xpan_beta.factory_coefs(xalpha=False, central=False)
# data object
data = xpan_beta.factory_data(u, x, order=order, chunk=1000)

In [11]:
data.xu

<xarray.DataArray 'x' (moment: 7, val: 5)>
array([[0.50041065, 0.49899555, 0.50038186, 0.49912602, 0.50115189],
       [0.25009013, 0.24913469, 0.25014455, 0.2493188 , 0.25043612],
       [0.16679052, 0.16610542, 0.1668418 , 0.16617908, 0.16705   ],
       [0.1251341 , 0.12463769, 0.1251815 , 0.12467509, 0.12534457],
       [0.10011993, 0.0997598 , 0.10017438, 0.0997978 , 0.10030181],
       [0.08343107, 0.08317153, 0.08349794, 0.08322209, 0.08359674],
       [0.07150348, 0.07131927, 0.07158309, 0.07138435, 0.0716602 ]])
Dimensions without coordinates: moment, val

In [22]:
data.u

<xarray.DataArray 'u' (moment: 7)>
array([1.        , 0.49950003, 0.33304221, 0.24985064, 0.19992672,
       0.16663601, 0.14285173])
Dimensions without coordinates: moment

In [13]:
import cmomy.xcentral as xcentral

(100000, 5)

In [30]:
ave_u = xcentral.xStatsAccum.from_vals(u, moments=order)

In [48]:
ave_xu = xcentral.xStatsAccumCov.from_vals(x, u, broadcast=True, moments=(1, order), dims=['val'])

In [49]:
ave_u.to_raw().rename({'mom_0':'moment'})

<xarray.DataArray (moment: 7)>
array([1.00000000e+05, 4.99500033e-01, 3.33042206e-01, 2.49850644e-01,
       1.99926724e-01, 1.66636005e-01, 1.42851727e-01])
Dimensions without coordinates: moment

In [52]:
ave_xu.to_raw().sel(mom_0=1).transpose()

<xarray.DataArray (mom_1: 7, val: 5)>
array([[0.50041065, 0.49899555, 0.50038186, 0.49912602, 0.50115189],
       [0.25009013, 0.24913469, 0.25014455, 0.2493188 , 0.25043612],
       [0.16679052, 0.16610542, 0.1668418 , 0.16617908, 0.16705   ],
       [0.1251341 , 0.12463769, 0.1251815 , 0.12467509, 0.12534457],
       [0.10011993, 0.0997598 , 0.10017438, 0.0997978 , 0.10030181],
       [0.08343107, 0.08317153, 0.08349794, 0.08322209, 0.08359674],
       [0.07150348, 0.07131927, 0.07158309, 0.07138435, 0.0716602 ]])
Dimensions without coordinates: mom_1, val

In [51]:
data.xu

<xarray.DataArray 'x' (moment: 7, val: 5)>
array([[0.50041065, 0.49899555, 0.50038186, 0.49912602, 0.50115189],
       [0.25009013, 0.24913469, 0.25014455, 0.2493188 , 0.25043612],
       [0.16679052, 0.16610542, 0.1668418 , 0.16617908, 0.16705   ],
       [0.1251341 , 0.12463769, 0.1251815 , 0.12467509, 0.12534457],
       [0.10011993, 0.0997598 , 0.10017438, 0.0997978 , 0.10030181],
       [0.08343107, 0.08317153, 0.08349794, 0.08322209, 0.08359674],
       [0.07150348, 0.07131927, 0.07158309, 0.07138435, 0.0716602 ]])
Dimensions without coordinates: moment, val

In [23]:
out = xcentral.xStatsAccumCov.from_vals(x, u, broadcast=True, moments=(order, 1), dims=['val'])

In [29]:
out.values.sel(mom_0=0)

<xarray.DataArray (val: 5, mom_1: 2)>
array([[1.00000000e+05, 4.99500033e-01],
       [1.00000000e+05, 4.99500033e-01],
       [1.00000000e+05, 4.99500033e-01],
       [1.00000000e+05, 4.99500033e-01],
       [1.00000000e+05, 4.99500033e-01]])
Dimensions without coordinates: val, mom_1

In [13]:
# Note very slight difference in averages.
# this is due to numerics
print(data.u - [ufunc(i) for i in range(order+1)])
print(data.xu - [xufunc(i) for i in range(order+1)])

<xarray.DataArray 'u' (moment: 7)>
array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       5.55111512e-17, 2.77555756e-17, 0.00000000e+00])
Dimensions without coordinates: moment
<xarray.DataArray 'x' (moment: 7, val: 5)>
array([[ 1.11022302e-15,  6.66133815e-16, -3.33066907e-16,
         6.77236045e-15, -4.77395901e-15],
       [-1.49880108e-15, -3.88578059e-16,  0.00000000e+00,
        -5.55111512e-17, -1.27675648e-15],
       [ 2.77555756e-17, -1.99840144e-15,  1.44328993e-15,
         1.05471187e-15,  2.49800181e-16],
       [ 4.71844785e-16,  9.43689571e-16,  1.44328993e-15,
         1.80411242e-16, -7.21644966e-16],
       [ 1.66533454e-15, -1.04083409e-15,  1.33226763e-15,
        -7.63278329e-16,  1.98452366e-15],
       [ 3.74700271e-16, -4.85722573e-16,  4.30211422e-16,
        -1.20736754e-15, -2.77555756e-17],
       [ 2.77555756e-16,  6.52256027e-16,  4.16333634e-17,
         2.91433544e-16,  8.32667268e-16]])
Dimensions without coordinates: moment, v

## coefficients of expansion

In [14]:
# norm = True, include 1/ n!  factor
# if want straight coeffs, use norm=False
c = s.xcoefs(data, norm=False) 
c

<xarray.DataArray 'x' (order: 7, val: 5)>
array([[ 5.00410649e-01,  4.98995550e-01,  5.00381858e-01,
         4.99126019e-01,  5.01151890e-01],
       [-1.34997699e-04,  1.13607821e-04, -2.03796624e-04,
        -5.34078264e-06, -1.10735336e-04],
       [-2.20835067e-06,  3.23344629e-05, -1.00746807e-05,
        -5.62857597e-05,  3.46399234e-05],
       [ 2.53920723e-05, -2.83855528e-05,  2.77531114e-05,
        -4.71421307e-05,  3.10969286e-05],
       [-5.30174199e-06, -1.05343641e-05,  6.58390419e-06,
         2.21207532e-05, -9.60663726e-06],
       [-1.30147200e-05,  1.45382620e-05, -1.37400869e-05,
         2.72303311e-05, -1.71110507e-05],
       [ 4.96909742e-06,  7.86225090e-06, -6.25406039e-06,
        -1.76766305e-05,  7.29034558e-06]])
Dimensions without coordinates: order, val

In [15]:
# test equal to old way
np.testing.assert_allclose(c, coefs_list, rtol=1e-5)

In [17]:
# instead, can use central moments
s_c = xpan_beta.factory_coefs(central=True)

In [18]:
data_c = xpan_beta.factory_data(u, x, order=order, central=True)

c_c = s_c.xcoefs(data_c, norm=False)
c_c

<xarray.DataArray 'x' (order: 7, val: 5)>
array([[ 5.00410649e-01,  4.98995550e-01,  5.00381858e-01,
         4.99126019e-01,  5.01151890e-01],
       [-1.34997699e-04,  1.13607821e-04, -2.03796624e-04,
        -5.34078264e-06, -1.10735336e-04],
       [-2.20835067e-06,  3.23344629e-05, -1.00746807e-05,
        -5.62857597e-05,  3.46399234e-05],
       [ 2.53920723e-05, -2.83855528e-05,  2.77531114e-05,
        -4.71421307e-05,  3.10969286e-05],
       [-5.30174199e-06, -1.05343641e-05,  6.58390419e-06,
         2.21207532e-05, -9.60663726e-06],
       [-1.30147200e-05,  1.45382620e-05, -1.37400869e-05,
         2.72303311e-05, -1.71110507e-05],
       [ 4.96909742e-06,  7.86225090e-06, -6.25406039e-06,
        -1.76766305e-05,  7.29034558e-06]])
Dimensions without coordinates: order, val

In [19]:
# test close
np.testing.assert_allclose(s.xcoefs(data), s_c.xcoefs(data_c))


# nice-ness of central moments

In [20]:
# coeffs using raw moments u[i], xu[i]
for i in range(order+1):
    print('{}: {}'.format(i, s.exprs[i]))

0: xu[0]
1: u[1]*xu[0] - xu[1]
2: 2*u[1]**2*xu[0] - 2*u[1]*xu[1] - u[2]*xu[0] + xu[2]
3: 6*u[1]**3*xu[0] - 6*u[1]**2*xu[1] - 6*u[1]*u[2]*xu[0] + 3*u[1]*xu[2] + 3*u[2]*xu[1] + u[3]*xu[0] - xu[3]
4: 24*u[1]**4*xu[0] - 24*u[1]**3*xu[1] - 36*u[1]**2*u[2]*xu[0] + 12*u[1]**2*xu[2] + 24*u[1]*u[2]*xu[1] + 8*u[1]*u[3]*xu[0] - 4*u[1]*xu[3] + 6*u[2]**2*xu[0] - 6*u[2]*xu[2] - 4*u[3]*xu[1] - u[4]*xu[0] + xu[4]
5: 120*u[1]**5*xu[0] - 120*u[1]**4*xu[1] - 240*u[1]**3*u[2]*xu[0] + 60*u[1]**3*xu[2] + 180*u[1]**2*u[2]*xu[1] + 60*u[1]**2*u[3]*xu[0] - 20*u[1]**2*xu[3] + 90*u[1]*u[2]**2*xu[0] - 60*u[1]*u[2]*xu[2] - 40*u[1]*u[3]*xu[1] - 10*u[1]*u[4]*xu[0] + 5*u[1]*xu[4] - 30*u[2]**2*xu[1] - 20*u[2]*u[3]*xu[0] + 10*u[2]*xu[3] + 10*u[3]*xu[2] + 5*u[4]*xu[1] + u[5]*xu[0] - xu[5]
6: 720*u[1]**6*xu[0] - 720*u[1]**5*xu[1] - 1800*u[1]**4*u[2]*xu[0] + 360*u[1]**4*xu[2] + 1440*u[1]**3*u[2]*xu[1] + 480*u[1]**3*u[3]*xu[0] - 120*u[1]**3*xu[3] + 1080*u[1]**2*u[2]**2*xu[0] - 540*u[1]**2*u[2]*xu[2] - 360*u[1]**2*u[3]*xu[1]

In [21]:
# coeffs using central moments du[i] = <(u-<u>)**i> 
for i in range(order+1):
    print('{}: {}'.format(i, s_c.exprs[i]))

0: x1
1: -dxdu[1]
2: dxdu[2]
3: 3*du[2]*dxdu[1] - dxdu[3]
4: -6*du[2]*dxdu[2] - 4*du[3]*dxdu[1] + dxdu[4]
5: -30*du[2]**2*dxdu[1] + 10*du[2]*dxdu[3] + 10*du[3]*dxdu[2] + 5*du[4]*dxdu[1] - dxdu[5]
6: 90*du[2]**2*dxdu[2] + 120*du[2]*du[3]*dxdu[1] - 15*du[2]*dxdu[4] - 20*du[3]*dxdu[3] - 15*du[4]*dxdu[2] - 6*du[5]*dxdu[1] + dxdu[6]


In [22]:
# so not only are central moments more stable, the resulting expressions
# are way simplier!

# taking advantage of xarray

In [23]:
# make a dataset of two values:

x_set = xr.merge((core.xrwrap_xv(xx, name=name) 
                  for xx, name in zip([x, xb], ['a','b'])))
print(x_set)

data_set = xpan_beta.factory_data(u, x_set, order)
data_set_c = xpan_beta.factory_data(u, x_set, order, central=True)


# need "other" x for comparison
datab = xpan_beta.factory_data(u, xb, order)
datab_c = xpan_beta.factory_data(u, xb, order, central=True)

<xarray.Dataset>
Dimensions:  (rec: 100000, val: 5)
Dimensions without coordinates: rec, val
Data variables:
    a        (rec, val) float64 0.5353 0.904 0.5024 ... 0.7553 0.2138 0.7261
    b        (rec, val) float64 0.6237 0.1782 0.2007 ... 0.08856 0.0782 0.4235


In [24]:
# note that data_set has multiple measurements in it.  Very nice...
print(data_set.xv)
print(data_set.xu)

<xarray.Dataset>
Dimensions:  (rec: 100000, val: 5)
Dimensions without coordinates: rec, val
Data variables:
    a        (rec, val) float64 0.5353 0.904 0.5024 ... 0.7553 0.2138 0.7261
    b        (rec, val) float64 0.6237 0.1782 0.2007 ... 0.08856 0.0782 0.4235
<xarray.Dataset>
Dimensions:  (moment: 7, val: 5)
Dimensions without coordinates: moment, val
Data variables:
    a        (moment, val) float64 0.5004 0.499 0.5004 ... 0.07138 0.07166
    b        (moment, val) float64 0.5001 0.5015 0.5001 ... 0.07127 0.07149


In [25]:
ufuncb, xufuncb = thermoextrap.buildAvgFuncs(xb, u, order)
coefs_listb = [fs[i](ufuncb, xufuncb) for i in range(order+1)]

In [26]:
# make sure have same value
np.testing.assert_allclose(s.xcoefs(datab,  norm=False), coefs_listb)

In [27]:
# consider dataset
c_set = s.xcoefs(data_set, norm=False)
c_set

<xarray.Dataset>
Dimensions:  (order: 7, val: 5)
Dimensions without coordinates: order, val
Data variables:
    a        (order, val) float64 0.5004 0.499 0.5004 ... -1.768e-05 7.29e-06
    b        (order, val) float64 0.5001 0.5015 0.5001 ... -2.571e-06 1.063e-05

In [28]:
# testing
cb = s.xcoefs(datab, norm=False)
np.testing.assert_allclose(c_set.a, c)
np.testing.assert_allclose(c_set.b, cb)

In [29]:
# central
c_set_c = s_c.xcoefs(data_set_c, norm=False)
# test
xr.testing.assert_allclose(c_set, c_set_c)

# resampling

In [30]:
# resampling
data.resample(10).u

<xarray.DataArray 'u' (moment: 7, rep: 10)>
array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [0.50010684, 0.49964365, 0.49834588, 0.49995062, 0.50089883,
        0.49890858, 0.50064861, 0.49993096, 0.498588  , 0.50043063],
       [0.3338034 , 0.33315953, 0.33196643, 0.33397058, 0.3344147 ,
        0.3325299 , 0.33430399, 0.33355302, 0.33218533, 0.33375627],
       [0.25067365, 0.25006705, 0.2487919 , 0.2509836 , 0.25106604,
        0.24953805, 0.2511051 , 0.25039425, 0.24909806, 0.25041308],
       [0.20076379, 0.20027508, 0.19886161, 0.20113612, 0.20101023,
        0.19979342, 0.20110628, 0.20047781, 0.19925382, 0.20039194],
       [0.16746092, 0.16710698, 0.16556748, 0.16785829, 0.16762127,
        0.16664629, 0.16770943, 0.16718061, 0.16601802, 0.16703544],
       [0.14365099, 0.14342314, 0.14178795, 0.14405387, 0.14376448,
        0.14297293, 0.14381061, 0.14338245, 0.14227237, 0.14320367]])
Dimensions without coordinates: moment, rep

In [53]:
# testing resampling:
# use same resampler for all
idx = core.resample_indicies(len(data), nrep=10)

In [54]:
idx.shape

(100000, 10)

In [32]:
c_r = s.xcoefs(data.resample(nrep=None, idx=idx))
cb_r = s.xcoefs(datab.resample(None, idx))
cs_r = s.xcoefs(data_set.resample(None, idx))

In [33]:
np.testing.assert_allclose(cs_r.a, c_r)
np.testing.assert_allclose(cs_r.b, cb_r)

In [34]:
# center
cs_c_r = s_c.xcoefs(data_set_c.resample(None, idx))

In [35]:
xr.testing.assert_allclose(cs_r, cs_c_r)

# resampling time and chunking

In [36]:
# xarray is built on dask.  I'm no expert, that this speed things up
# use chunking to create a dask array of uv, xv

In [37]:
idx = core.resample_indicies(len(data), nrep=100)

In [38]:
%%timeit -n 1 -r 1
r = data.resample(None, idx, chunk=None)
r.u
r.xu

2.11 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [39]:
%%timeit -n 1 -r 1
r = data.resample(None, idx, chunk=10000)
r.u
r.xu

1.73 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Extrap model

In [10]:
reload(core)

<module 'thermoextrap.xtrapy.core' from '/Users/wpk/Documents/python/projects/xtrapy/thermoextrap/xtrapy/core.py'>

In [6]:
# test Extrap model
betas = [0.3, 0.4]
ref_beta = 0.5

In [42]:
# blank model
em = thermoextrap.ExtrapModel(maxOrder=order)

In [43]:
params = em.train(ref_beta, xData=x, uData=u, saveParams=True)

In [44]:
em.predict(betas, order=3)

array([[0.50043757, 0.49897351, 0.50042238, 0.49912602, 0.50117469],
       [0.50042413, 0.49898436, 0.50040218, 0.49912628, 0.50116313]])

In [13]:
xem = xpan_beta.factory_extrapmodel(order, ref_beta, u, x, xalpha=False, central=False)
xem.predict(betas, order=3)

<xarray.DataArray (beta: 2, val: 5)>
array([[0.50043757, 0.49897351, 0.50042238, 0.49912602, 0.50117469],
       [0.50042413, 0.49898436, 0.50040218, 0.49912628, 0.50116313]])
Coordinates:
  * beta     (beta) float64 0.3 0.4
    dbeta    (beta) float64 -0.2 -0.1
    beta0    float64 0.5
Dimensions without coordinates: val

In [14]:
# central, and build from "from_values" method
xem_c = xpan_beta.factory_extrapmodel(order, ref_beta, u, x, xalpha=False, central=True)
xem_c.predict(betas, order=3)

<xarray.DataArray (beta: 2, val: 5)>
array([[0.50043757, 0.49897351, 0.50042238, 0.49912602, 0.50117469],
       [0.50042413, 0.49898436, 0.50040218, 0.49912628, 0.50116313]])
Coordinates:
  * beta     (beta) float64 0.3 0.4
    dbeta    (beta) float64 -0.2 -0.1
    beta0    float64 0.5
Dimensions without coordinates: val

In [47]:
# resample
em.bootstrap(betas, n=20)

array([[0.00106026, 0.00092145, 0.00066888, 0.00095758, 0.00098947],
       [0.00105866, 0.0009081 , 0.00067315, 0.00095227, 0.00098661]])

In [15]:
# resample data
xem_r = xem.resample(20)
xem_r.predict(betas).std('rep')

<xarray.DataArray (beta: 2, val: 5)>
array([[0.00104206, 0.00091728, 0.00099767, 0.00106403, 0.00086129],
       [0.00104536, 0.00091457, 0.00099733, 0.0010722 , 0.00086071]])
Coordinates:
  * beta     (beta) float64 0.3 0.4
    dbeta    (beta) float64 -0.2 -0.1
    beta0    float64 0.5
Dimensions without coordinates: val

In [16]:
xem_c.resample(20).predict(betas).std('rep')

<xarray.DataArray (beta: 2, val: 5)>
array([[0.00109224, 0.00079184, 0.00096688, 0.00101369, 0.00075356],
       [0.00109342, 0.00079328, 0.00098064, 0.00101566, 0.00074715]])
Coordinates:
  * beta     (beta) float64 0.3 0.4
    dbeta    (beta) float64 -0.2 -0.1
    beta0    float64 0.5
Dimensions without coordinates: val

In [17]:
# note that xem_r is just an Extrap model with new resampled data
xem_r.data.u

<xarray.DataArray 'u' (moment: 7, rep: 20)>
array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [0.50010684, 0.49964365, 0.49834588, 0.49995062, 0.50089883,
        0.49890858, 0.50064861, 0.49993096, 0.498588  , 0.50043063,
        0.50016971, 0.50091015, 0.4987761 , 0.50052337, 0.49910457,
        0.50008106, 0.49954187, 0.49916412, 0.49978279, 0.49850368],
       [0.3338034 , 0.33315953, 0.33196643, 0.33397058, 0.3344147 ,
        0.3325299 , 0.33430399, 0.33355302, 0.33218533, 0.33375627,
        0.3335511 , 0.33394193, 0.33240869, 0.33431928, 0.3324881 ,
        0.33346312, 0.3332114 , 0.33236782, 0.33313493, 0.33226926],
       [0.25067365, 0.25006705, 0.2487919 , 0.2509836 , 0.25106604,
        0.24953805, 0.2511051 , 0.25039425, 0.24909806, 0.25041308,
        0.25020343, 0.25029616, 0.24939816, 0.25119668, 0.249256  ,
        0.25003906, 0.25005408, 0.24906682, 0.24980715, 0.24924002],
       [0.20076379, 0.20027508, 0.19886161, 0.20113612, 0.20101023,
        0.19979342, 0.20110628, 0.20047781, 0.19925382, 0.20039194,
        0.20016312, 0.20007018, 0.19962362, 0.20126201, 0.19934899,
        0.19991272, 0.20012233, 0.19912504, 0.19979536, 0.19941441],
       [0.16746092, 0.16710698, 0.16556748, 0.16785829, 0.16762127,
        0.16664629, 0.16770943, 0.16718061, 0.16601802, 0.16703544,
        0.16678751, 0.16659042, 0.16643759, 0.16792291, 0.16610253,
        0.16646937, 0.16681294, 0.16585047, 0.16644912, 0.16618854],
       [0.14365099, 0.14342314, 0.14178795, 0.14405387, 0.14376448,
        0.14297293, 0.14381061, 0.14338245, 0.14227237, 0.14320367,
        0.14294192, 0.14269141, 0.14272135, 0.14407351, 0.14237238,
        0.14257828, 0.1430108 , 0.14209358, 0.14263277, 0.1424512 ]])
Dimensions without coordinates: moment, rep

In [51]:
# testing dataset
emb = thermoextrap.ExtrapModel(order, ref_beta, xb, u)

In [52]:
xemb = xpan_beta.factory_extrapmodel(order, ref_beta, u, xb, central=True)
xem_set = xpan_beta.factory_extrapmodel(order, ref_beta, u, x_set, central=True)

In [53]:
np.testing.assert_allclose(emb.predict(betas), emb.predict(betas))

In [54]:
# testing dataset
out = xem.predict(betas)
outb = xemb.predict(betas)
outs = xem_set.predict(betas)

np.testing.assert_allclose(out, outs.a)
np.testing.assert_allclose(outb, outs.b)

In [55]:
# resampling
idx = core.resample_indicies(len(x), 20)

out = xem.resample(None, idx).predict(betas)
outb = xemb.resample(None, idx).predict(betas)
outs = xem_set.resample(None, idx).predict(betas)

In [56]:
np.testing.assert_allclose(out, outs.a)
np.testing.assert_allclose(outb, outs.b)

In [57]:
# resampling much faster with chunking

In [58]:
%%timeit -n 1 -r 1
print(em.bootstrap(betas))

[[0.00086844 0.00088233 0.00096163 0.00101373 0.00084704]
 [0.00086735 0.00088198 0.0009599  0.0010106  0.0008449 ]]
5.05 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [59]:
%%timeit -n 1 -r 1
print(xem.resample(100, chunk=1000).predict(betas).std('rep'))

<xarray.DataArray (alpha: 2, val: 5)>
array([[0.00087964, 0.00088115, 0.0009716 , 0.00092574, 0.00096312],
       [0.00087682, 0.000882  , 0.00097072, 0.00092256, 0.00096504]])
Coordinates:
  * alpha    (alpha) float64 0.3 0.4
    dalpha   (alpha) float64 -0.2 -0.1
    alpha0   float64 0.5
Dimensions without coordinates: val
1.99 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# ExtrapWeighted

In [18]:
u = np.random.rand(2,n) + 5
x = np.random.rand(2,n, nv) + 10
xb = np.random.rand(2,n, 5) + 2
order = 4

In [19]:
beta0 = [0.05, 0.5]
betas = [0.1, 0.2, 0.3, 0.4]

In [33]:
emw = thermoextrap.ExtrapWeightedModel(order, beta0, x, u)
emwB = thermoextrap.ExtrapWeightedModel(order, beta0, xb, u)

In [34]:
# for xtrapy, create list of models for each state:

# first create datasets for xs
xs = [xr.Dataset({'a': core.xrwrap_xv(xx), 
                  'b': core.xrwrap_xv(xxb)})
      for xx, xxb in zip(x, xb)
     ]

# now crete list of models for each state
states = [xpan_beta.factory_extrapmodel(order, b0, uu, xx, central=True)
         for b0, uu, xx in zip(beta0, u, xs)]

xemw = core.ExtrapWeightedModel(states)

In [35]:
states_a = [xpan_beta.factory_extrapmodel(order, b0, uu, xx, central=True)
             for b0, uu, xx in zip(beta0, u, x)]
xemw_a = core.ExtrapWeightedModel(states_a)

In [37]:
np.testing.assert_allclose(emw.predict(betas), xemw.predict(betas).a)
np.testing.assert_allclose(emwB.predict(betas), xemw.predict(betas).b)

In [38]:
idxs = [core.resample_indicies(len(xemw[0].data), 20) for i in range(2)]
a = xemw.resample(None, idxs).predict(betas).a
b = xemw_a.resample(None, idxs).predict(betas)

In [39]:
%%timeit -n 1 -r 1
emw.bootstrap(betas, n=100)

6.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [72]:
%%timeit -n 1 -r 1
# note that is is actually doing twice the work
# because data includes both x and xb
xemw.resample(100).predict(betas).std('rep')

7.37 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [73]:
%%timeit -n 1 -r 1
# note that is is actually doing twice the work
# because data includes both x and xb
xemw.resample(100, chunk=1000).predict(betas).std('rep')

6.18 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [74]:
%%timeit -n 1 -r 1
# note that is is actually doing twice the work
# because data includes both x and xb
xemw_a.resample(100, chunk=10000).predict(betas).std('rep')

3.26 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [70]:
from dask.diagnostics import ProgressBar

In [71]:
%%timeit -n 1 -r 1
# note that is is actually doing twice the work
# because data includes both x and xb
with ProgressBar():
    xemw_a.resample(500, chunk=10000).predict(betas).std('rep')

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.6s
[########################################] | 100% Completed |  4.6s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.7s
[########################################] | 100% Completed |  2.5s
22.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# InterpModel

In [40]:
emi = thermoextrap.InterpModel(order, beta0, x, u)
emib = thermoextrap.InterpModel(order, beta0, xb, u)

In [41]:
xemi = core.InterpModel(states)

In [42]:
states_b = [xpan_beta.factory_extrapmodel(order, b0, uu, xx, central=True)
             for b0, uu, xx in zip(beta0, u, xb)]

xemi_a = core.InterpModel(states_a)
xemi_b = core.InterpModel(states_b)


In [43]:
alpha = [0.1, 0.2]

In [44]:
np.testing.assert_allclose(emi.predict(betas), xemi.predict(betas).a)
np.testing.assert_allclose(emib.predict(betas), xemi.predict(betas).b)

In [45]:
emi.bootstrap(betas, n=20)

array([[0.00073394, 0.00082176, 0.00086818, 0.0007307 , 0.00098333],
       [0.00064477, 0.00069775, 0.00075103, 0.00060203, 0.00087918],
       [0.00067547, 0.00045663, 0.00084683, 0.00053983, 0.00074061],
       [0.00095478, 0.00059577, 0.00120691, 0.00084186, 0.0008883 ]])

In [46]:
xemi.resample(20).predict(betas).std('rep').a

<xarray.DataArray 'a' (beta: 4, val: 5)>
array([[0.00091733, 0.0009824 , 0.00084456, 0.0010354 , 0.00095344],
       [0.00075589, 0.00082176, 0.00070198, 0.00086866, 0.00083223],
       [0.00062099, 0.00054772, 0.00057055, 0.0006534 , 0.00068517],
       [0.0009579 , 0.00076648, 0.00086728, 0.00092517, 0.0008929 ]])
Coordinates:
  * beta     (beta) float64 0.1 0.2 0.3 0.4
Dimensions without coordinates: val

In [82]:
%%timeit -n 1 -r 1
emi.bootstrap(betas, n=100)

6.77 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [83]:
%%timeit -n 1 -r 1
xemi_a.resample(100, chunk=10000).predict(betas).std('rep')

3.25 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# MBAR

In [48]:
mbar = thermoextrap.MBARModel(refB=beta0, xData=x, uData=u)

In [49]:
mbar.predict(betas)

array([[10.49946905, 10.50009106, 10.49990332, 10.50020218, 10.50030235],
       [10.49945993, 10.50010948, 10.49990185, 10.50022119, 10.50031344],
       [10.49945098, 10.50012735, 10.49989977, 10.50024016, 10.50032404],
       [10.49944219, 10.50014468, 10.49989709, 10.50025908, 10.50033412]])

In [50]:
xmbar = core.MBARModel(states_a)

In [51]:
out = xmbar.predict(betas)

In [52]:
out

<xarray.DataArray (beta: 4, val: 5)>
array([[10.49946905, 10.50009106, 10.49990332, 10.50020218, 10.50030235],
       [10.49945993, 10.50010948, 10.49990185, 10.50022119, 10.50031344],
       [10.49945098, 10.50012735, 10.49989977, 10.50024016, 10.50032404],
       [10.49944219, 10.50014468, 10.49989709, 10.50025908, 10.50033412]])
Coordinates:
  * beta     (beta) float64 0.1 0.2 0.3 0.4
    alpha    (beta) float64 0.1 0.2 0.3 0.4
Dimensions without coordinates: val

In [53]:
np.testing.assert_allclose(mbar.predict(betas), xmbar.predict(betas))

# Perturb

In [54]:
reload(core)

<module 'thermoextrap.xtrapy.core' from '/Users/wpk/Documents/python/projects/xtrapy/thermoextrap/xtrapy/core.py'>

In [55]:
beta_ref = 0.5

In [56]:
pm = thermoextrap.PerturbModel(beta_ref, x[0], u[0])

In [57]:
pm.predict([0.1, 0.2], useMBAR=False)

array([[10.49888945, 10.49920956, 10.49992106, 10.4998412 , 10.4996373 ],
       [10.4988958 , 10.49927884, 10.49991802, 10.49984427, 10.49963441]])

In [58]:
xpm = xpan_beta.factory_perturbmodel(beta_ref, u[0], x[0])

In [59]:
xpm.predict([0.1, 0.2])

<xarray.DataArray (beta: 2, val: 5)>
array([[10.49888945, 10.49920956, 10.49992106, 10.4998412 , 10.4996373 ],
       [10.4988958 , 10.49927884, 10.49991802, 10.49984427, 10.49963441]])
Coordinates:
  * beta     (beta) float64 0.1 0.2
Dimensions without coordinates: val

# log func

In [60]:
from thermoextrap.utilities import buildAvgFuncs

#For quantities like the chemical potential, we're interested in the -log(<X>), not <X>
#Everything is the same, but we take derivatives differently
#Luckily, have closed-form expression for derivatives of -log(<X>) in terms of derivatives of <X>
#Specifically, d(n)[-log(<X>)]/dB(n) = Sum(k=1, n)[(k-1)! * (-1/<X>)^k * B(n,k,(d<X>/dB, ..., d(n-k+1)<X>/dB(n-k+1)))]
#B(n,k, (...)) represents Bell Polynomials, which are implemented in sympy (not numpy or scipy unfortunately)
#Create custom classes to handle this
from sympy import bell

class LogAvgExtrapModel(ExtrapModel):
    
    def calcDerivVals(self, refB, x, U):
        
        if x.shape[0] != U.shape[0]:
            print('First observable dimension (%i) and size of potential energy array (%i) do not match!'%(x.shape[0], U.shape[0]))
            return
    
        avgUfunc, avgXUfunc = buildAvgFuncs(x, U, self.maxOrder)
        derivVals = np.zeros((self.maxOrder+1, x.shape[1]))
        for o in range(self.maxOrder+1):
            if o == 0:
                derivVals[o] = (-np.log(avgXUfunc(0)))
                continue
            for k in range(1,o+1):
                #Get the derivatives of the average quantity
                thisDiffs = np.array([self.derivF[l](avgUfunc, avgXUfunc) for l in range(1, o-k+2)])
                #Loop to apply the chain rule to each element of the observable array
                for l in range(x.shape[1]):
                    derivVals[o,l] += np.math.factorial(k-1)*((-1/avgXUfunc(0)[l])**k)*bell(o, k, thisDiffs[:,l])
                
        return derivVals


In [61]:
betas

[0.1, 0.2, 0.3, 0.4]

In [62]:
xdata = xem.data.xv.values
udata = xem.data.uv.values
refBeta=0.5

In [63]:
#Create and train extrapolation model
extModelLog = LogAvgExtrapModel(maxOrder=4, refB=refBeta, 
                                xData=xdata,
                                uData=udata,
                                )

#Note that we handled the -log calculation in the definition of the derivatives (even at zeroth order).
#This means we want to just pass data, not the -log of the data.

#Check the parameters
print("Model parameters (derivatives):")
print(extModelLog.params)
print('\n')

#Finally, look at predictions
print("Model predictions:")
print(extModelLog.predict(betas, order=2))
print('\n')

Model parameters (derivatives):
[[ 6.92326220e-01  6.95158101e-01  6.92383757e-01  6.94896673e-01
   6.90846049e-01]
 [ 2.69773834e-04 -2.27673014e-04  4.07282201e-04  1.07002690e-05
   2.20961626e-04]
 [ 4.48585482e-06 -6.47472658e-05  2.02998635e-05  1.12768749e-04
  -6.90717840e-05]
 [-5.07388591e-05  5.69296180e-05 -5.54391284e-05  9.44529753e-05
  -6.20967030e-05]
 [ 1.05400889e-05  2.10718898e-05 -1.32468610e-05 -4.42767814e-05
   1.91285621e-05]]


Model predictions:
[[0.69221867 0.69524399 0.69222247 0.69490141 0.69075214]
 [0.69224549 0.69522349 0.69226249 0.69489854 0.69077665]
 [0.69227236 0.69520234 0.69230271 0.69489679 0.69080048]
 [0.69229927 0.69518054 0.69234313 0.69489617 0.69082361]]




In [64]:
xem_log = xpan_beta.factory_extrapmodel(4, refBeta, udata, xdata, 
                                            xalpha=False, central=False, minus_log=True)

In [65]:
xem_log.xcoefs(norm=False) - extModelLog.params

<xarray.DataArray 'x' (order: 5, val: 5)>
array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 8.47032947e-22, -1.35525272e-20, -5.54637174e-17,
         1.35525272e-20,  0.00000000e+00],
       [ 5.54637174e-17, -5.56195714e-17, -5.55450325e-17,
        -2.78030095e-16, -1.35525272e-20],
       [ 3.88318867e-16,  8.34835673e-17,  3.32712848e-16,
         2.22417299e-16,  8.30756362e-16]])
Dimensions without coordinates: order, val

In [66]:
xem_log.predict(betas, order=3) - extModelLog.predict(betas, order=3)

<xarray.DataArray (beta: 4, val: 5)>
array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.11022302e-16,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00]])
Coordinates:
  * beta     (beta) float64 0.1 0.2 0.3 0.4
    dalpha   (beta) float64 -0.4 -0.3 -0.2 -0.1
    beta0    float64 0.5
Dimensions without coordinates: val

In [67]:
%%timeit -n 1 -r 1
print(xem_log.resample(100).predict(betas,order=2).std('rep'))

<xarray.DataArray (beta: 4, val: 5)>
array([[0.00097982, 0.00096949, 0.00089208, 0.0008623 , 0.0008552 ],
       [0.00097688, 0.0009669 , 0.00089064, 0.00086231, 0.00085347],
       [0.00097458, 0.00096483, 0.00088979, 0.00086318, 0.00085273],
       [0.00097292, 0.00096326, 0.00088954, 0.00086491, 0.00085297]])
Coordinates:
  * beta     (beta) float64 0.1 0.2 0.3 0.4
    dalpha   (beta) float64 -0.4 -0.3 -0.2 -0.1
    beta0    float64 0.5
Dimensions without coordinates: val
1.91 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [68]:
%%timeit -n 1 -r 1
#And bootstrapped uncertainties
print("Bootstrapped uncertainties in predictions:")
print(extModelLog.bootstrap(betas, order=2))

Bootstrapped uncertainties in predictions:
[[0.00192916 0.00164824 0.00180508 0.00168583 0.00181047]
 [0.00192294 0.00164835 0.00180356 0.00167088 0.00180307]
 [0.0019179  0.0016503  0.00180392 0.0016578  0.00179714]
 [0.00191404 0.00165404 0.00180615 0.00164661 0.00179271]]
7.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
